# Top-Down Camera Features 

1. Y_Sort Camera : a camera that places the elements to top of each other . That gives us nice fake 3D effect .
2. A camera that always centers the player . 
3. it is a camera box . A camera box basicly moves if the character goes in that way without centering the character. 
4. Controlling the camera with the keyboard 
5. Using mouse control to move the camera  
6. A camera that allows us to zoom 

__NOTE__ :Different kind of cameras can work with each other.

### Coding template of the project : 

In [1]:
import math
import pygame 
import random  
import time 

pygame.init()  
WIDTH , HEIGHT = 700 , 700
screen = pygame.display.set_mode((WIDTH , HEIGHT))  
clock = pygame.time.Clock()  
font = pygame.font.Font(None , 30)

def debug(info , y = 10 , x = 10):  
    display_surface = pygame.display.get_surface() 
    debug_surf = font.render(str(info) , True , 'White') 
    debug_rect = debug_surf.get_rect(topleft = (x,y) ) 
    pygame.draw.rect(display_surface , 'Black' , debug_rect)  
    display_surface.blit(debug_surf , debug_rect) 

BLACK = (0,0,0) 
WHITE = (255,255,255) 
RED = (255,0,0) 
GREEN = (0,255,0)  
LIGHT_GREEN = (30,204,102)
PURPLE = (204 , 30 , 102) 
LIGHT_BLUE = (30,102,204) 
DARK_BLUE = (0,0,51) 

class Tree(pygame.sprite.Sprite): 
    def __init__(self, pos_x , pos_y) -> None:
        super().__init__() 
        self.image = pygame.image.load("img/tree.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 

class Player(pygame.sprite.Sprite): 
    def __init__(self , camera_group ,  pos_x = 300 , pos_y = 300 , speed = 200) -> None:
        super().__init__() 
        self.image = pygame.image.load("img/player.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 
        self.direction = pygame.math.Vector2() 
        self.speed = speed 

    def input(self): 
        keys = pygame.key.get_pressed() 
        if keys[pygame.K_UP] : 
            self.direction.y = -1 
        
        elif keys[pygame.K_DOWN] : 
            self.direction.y = 1 

        else : 
            self.direction.y = 0 

        if keys[pygame.K_RIGHT] : 
            self.direction.x = 1 

        elif keys[pygame.K_LEFT]: 
            self.direction.x = -1 

        else : 
            self.direction.x = 0

    def update(self,delta_time) -> None: 
        self.input() 
        movement = self.speed * delta_time  
        debug(self.direction * movement ) 
        self.rect.center +=  self.direction * round(movement) 

        
camera_group = pygame.sprite.Group() 
player = Player(camera_group)  
camera_group.add(player)

BLOCK = 100
for _ in range(20): 
    pos_x = random.randint(BLOCK , WIDTH-BLOCK) 
    pos_y = random.randint(BLOCK , HEIGHT-BLOCK)  
    tree = Tree(pos_x , pos_y)  
    camera_group.add(tree)



previous_time = time.time()
running = True 
while running:  
    Dt = time.time() - previous_time  
    previous_time = time.time()
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT:

            exit() 
            pygame.quit() 
    
    screen.fill(LIGHT_BLUE)  
    camera_group.update(Dt) 
    camera_group.draw(screen)
    pygame.display.update()



pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

### Y-Sort Camera Implementation : 

<img src="img/Y_sort1.jpg" width="600px">

In [1]:
import math
import pygame 
import random  
import time 

pygame.init()  
WIDTH , HEIGHT = 700 , 700
screen = pygame.display.set_mode((WIDTH , HEIGHT))  
clock = pygame.time.Clock()  
font = pygame.font.Font(None , 30)

def debug(info , y = 10 , x = 10):  
    display_surface = pygame.display.get_surface() 
    debug_surf = font.render(str(info) , True , 'White') 
    debug_rect = debug_surf.get_rect(topleft = (x,y) ) 
    pygame.draw.rect(display_surface , 'Black' , debug_rect)  
    display_surface.blit(debug_surf , debug_rect) 

BLACK = (0,0,0) 
WHITE = (255,255,255) 
RED = (255,0,0) 
GREEN = (0,255,0)  
LIGHT_GREEN = (30,204,102)
PURPLE = (204 , 30 , 102) 
LIGHT_BLUE = (30,102,204) 
DARK_BLUE = (0,0,51) 

class Tree(pygame.sprite.Sprite): 
    def __init__(self, pos_x , pos_y , group) -> None:
        super().__init__(group) 
        self.image = pygame.image.load("img/tree.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 

class Player(pygame.sprite.Sprite): 
    def __init__(self , camera_group ,  pos_x = 300 , pos_y = 300 , speed = 200) -> None:
        super().__init__(camera_group) 
        self.image = pygame.image.load("img/player.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 
        self.direction = pygame.math.Vector2() 
        self.speed = speed 

    def input(self): 
        keys = pygame.key.get_pressed() 
        if keys[pygame.K_UP] : 
            self.direction.y = -1 
        
        elif keys[pygame.K_DOWN] : 
            self.direction.y = 1 

        else : 
            self.direction.y = 0 

        if keys[pygame.K_RIGHT] : 
            self.direction.x = 1 

        elif keys[pygame.K_LEFT]: 
            self.direction.x = -1 

        else : 
            self.direction.x = 0

    def update(self,delta_time) -> None: 
        self.input() 
        movement = self.speed * delta_time  
        debug(self.direction * movement ) 
        self.rect.center +=  self.direction * round(movement) 

class Camera(pygame.sprite.Group): 
    def __init__(self) -> None:
        super().__init__()  
        self.display_surface = pygame.display.get_surface()  
        self.ground_surface = pygame.image.load("img/ground.png") 
        self.ground_rect = self.ground_surface.get_rect(topleft = (0,0))

    def custom_draw(self):  
        self.display_surface.blit(self.ground_surface , self.ground_rect)
        for sprite in sorted(self.sprites() , key = lambda sprite: sprite.rect.centery): 
            self.display_surface.blit(sprite.image , sprite.rect)  


        
camera_group = Camera()
player = Player(camera_group)  


BLOCK = 100
for _ in range(20): 
    pos_x = random.randint(BLOCK , WIDTH-BLOCK) 
    pos_y = random.randint(BLOCK , HEIGHT-BLOCK)  
    tree = Tree(pos_x , pos_y , camera_group)  



previous_time = time.time()
running = True 
while running:  
    Dt = time.time() - previous_time  
    previous_time = time.time()
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT:

            exit() 
            pygame.quit() 
    
    screen.fill(LIGHT_BLUE)  
    camera_group.update(Dt) 
    camera_group.custom_draw()
    pygame.display.update()

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

### Centering Camera 

Theory of centring camera is a bit confusing. But you can look at the code and try to understand it 

<img src="img/player_centered_camera.jpg" width="600px">

Camera is filming the rectangle that starts in the x=0 , y=0 cordinates . So we want it to go to some direction , that depends on the player's movement. If our aim is centering the camera , we can think of moving the objects to the direction which is the reverse of the players movement. (Think it like relative movement. When you go to left , you think the trees go to right) . So the offset that we are talking about should be negative vector of the player's movement.  

<img src="img/centering2.jpg" width="600px">

We can get the offset value with the representation that is given above. Normally the object's center should be `x = width//2` and `y= height//2` . But if the character decides to go to right , the character wouldnt be in the center of the screen . So we can create the offset vector depend on those facts :  

`offset_x = player.centerx - screen.half_width` 

`offset_y = player.centery - screen.half_height`  


ofsett values are 0 if the player doesnt move. But if it moves to right , offset_x becomes positive  . Then we do :
 
`new_ground_position = ground_rect.topleft - (offset_x , offset_y)` . e.g if you move 100 pixels right(+) , ground goes left to 100 px and you start to blit this ground image from (-100 , 0) instead of (0,0)   

(That is actually why we are subtracting the offset value) 
 



### Centering Camera Implementation : 

In [1]:
import math
import pygame 
import random  
import time 

pygame.init()  
WIDTH , HEIGHT = 700 , 700
screen = pygame.display.set_mode((WIDTH , HEIGHT))  
clock = pygame.time.Clock()  
font = pygame.font.Font(None , 30)

def debug(info , y = 10 , x = 10):  
    display_surface = pygame.display.get_surface() 
    debug_surf = font.render(str(info) , True , 'White') 
    debug_rect = debug_surf.get_rect(topleft = (x,y) ) 
    pygame.draw.rect(display_surface , 'Black' , debug_rect)  
    display_surface.blit(debug_surf , debug_rect) 

BLACK = (0,0,0) 
WHITE = (255,255,255) 
RED = (255,0,0) 
GREEN = (0,255,0)  
LIGHT_GREEN = (30,204,102)
PURPLE = (204 , 30 , 102) 
LIGHT_BLUE = (30,102,204) 
DARK_BLUE = (0,0,51) 

class Tree(pygame.sprite.Sprite): 
    def __init__(self, pos_x , pos_y , group) -> None:
        super().__init__(group) 
        self.image = pygame.image.load("img/tree.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 

class Player(pygame.sprite.Sprite): 
    def __init__(self , camera_group ,  pos_x = 300 , pos_y = 300 , speed = 200) -> None:
        super().__init__(camera_group) 
        self.image = pygame.image.load("img/player.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 
        self.direction = pygame.math.Vector2() 
        self.speed = speed 

    def input(self): 
        keys = pygame.key.get_pressed() 
        if keys[pygame.K_UP] : 
            self.direction.y = -1 
        
        elif keys[pygame.K_DOWN] : 
            self.direction.y = 1 

        else : 
            self.direction.y = 0 

        if keys[pygame.K_RIGHT] : 
            self.direction.x = 1 

        elif keys[pygame.K_LEFT]: 
            self.direction.x = -1 

        else : 
            self.direction.x = 0

    def update(self,delta_time) -> None: 
        self.input() 
        movement = self.speed * delta_time  
        debug(self.direction * movement ) 
        self.rect.center +=  self.direction * round(movement) 

class Camera(pygame.sprite.Group): 
    def __init__(self) -> None:
        super().__init__()  
        self.display_surface = pygame.display.get_surface()  
        self.ground_surface = pygame.image.load("img/ground.png") 
        self.ground_rect = self.ground_surface.get_rect(topleft = (0,0)) 

        self.offset = pygame.math.Vector2() 
        w , h = self.display_surface.get_size() 
        self.half_w , self.half_h = w//2 , h//2  

    def centering_camera(self , target : pygame.sprite.Sprite): 
        self.offset.x = target.rect.centerx - self.half_w 
        self.offset.y = target.rect.centery - self.half_h


    def custom_draw(self , center_target : pygame.sprite.Sprite):   

        self.centering_camera(center_target)
        ground_offset = self.ground_rect.topleft - self.offset 

        self.display_surface.blit(self.ground_surface , ground_offset)
        for sprite in sorted(self.sprites() , key = lambda sprite: sprite.rect.centery): 
            player_offset = sprite.rect.center - self.offset 
            self.display_surface.blit(sprite.image , player_offset)  


        
camera_group = Camera()
player = Player(camera_group)  


BLOCK = 100
for _ in range(20): 
    pos_x = random.randint(BLOCK , WIDTH-BLOCK) 
    pos_y = random.randint(BLOCK , HEIGHT-BLOCK)  
    tree = Tree(pos_x , pos_y , camera_group)  



previous_time = time.time()
running = True 
while running:  
    Dt = time.time() - previous_time  
    previous_time = time.time()
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT:

            exit() 
            pygame.quit() 
    
    screen.fill(LIGHT_BLUE)  
    camera_group.update(Dt) 
    camera_group.custom_draw(player)
    pygame.display.update()

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

### Camera Box 

Camera box model is used often in 2D Platformer games .  

> Just basicly taking the offset with the camera box while changing the camera box's position with the player if the player is going away from borders. 

<img src="img/camerabox1.jpg" width = "600px">

### Implementation of Camera Box Model

In [1]:
import math
import pygame 
import random  
import time 

pygame.init()  
WIDTH , HEIGHT = 700 , 700
screen = pygame.display.set_mode((WIDTH , HEIGHT))  
clock = pygame.time.Clock()  
font = pygame.font.Font(None , 30)

def debug(info , y = 10 , x = 10):  
    display_surface = pygame.display.get_surface() 
    debug_surf = font.render(str(info) , True , 'White') 
    debug_rect = debug_surf.get_rect(topleft = (x,y) ) 
    pygame.draw.rect(display_surface , 'Black' , debug_rect)  
    display_surface.blit(debug_surf , debug_rect) 

BLACK = (0,0,0) 
WHITE = (255,255,255) 
RED = (255,0,0) 
GREEN = (0,255,0)  
LIGHT_GREEN = (30,204,102)
PURPLE = (204 , 30 , 102) 
LIGHT_BLUE = (30,102,204) 
DARK_BLUE = (0,0,51) 

class Tree(pygame.sprite.Sprite): 
    def __init__(self, pos_x , pos_y , group) -> None:
        super().__init__(group) 
        self.image = pygame.image.load("img/tree.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 

class Player(pygame.sprite.Sprite): 
    def __init__(self , camera_group ,  pos_x = 350 , pos_y = 350 , speed = 200) -> None:
        super().__init__(camera_group) 
        self.image = pygame.image.load("img/player.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 
        self.direction = pygame.math.Vector2() 
        self.speed = speed 

    def input(self): 
        keys = pygame.key.get_pressed() 
        if keys[pygame.K_UP] : 
            self.direction.y = -1 
        
        elif keys[pygame.K_DOWN] : 
            self.direction.y = 1 

        else : 
            self.direction.y = 0 

        if keys[pygame.K_RIGHT] : 
            self.direction.x = 1 

        elif keys[pygame.K_LEFT]: 
            self.direction.x = -1 

        else : 
            self.direction.x = 0

    def update(self,delta_time) -> None: 
        self.input() 
        movement = self.speed * delta_time  
        debug(self.direction * movement ) 
        self.rect.center +=  self.direction * round(movement) 

class Camera(pygame.sprite.Group): 
    def __init__(self) -> None:
        super().__init__()  
        self.display_surface = pygame.display.get_surface()  
        self.ground_surface = pygame.image.load("img/ground.png") 
        self.ground_rect = self.ground_surface.get_rect(topleft = (0,0)) 

        self.offset = pygame.math.Vector2()


        screen_w , screen_h = self.display_surface.get_size() 
        self.borders = {"<":200 , ">" : 200 , "v" : 100 , "^":100} 
        l = self.borders[">"] 
        t = self.borders["^"] 
        w = screen_w - self.borders["<"] - self.borders[">"] 
        h = screen_h - self.borders["v"] - self.borders["^"] 
        self.cbox = pygame.Rect(l , t , w , h) 

        


    def center_camera_box(self,target: pygame.sprite.Sprite): 
        
        if target.rect.left < self.cbox.left : 
            self.cbox.left = target.rect.left  

        if target.rect.right > self.cbox.right : 
            self.cbox.right = target.rect.right 

        if target.rect.top < self.cbox.top : 
            self.cbox.top = target.rect.top 

        if target.rect.bottom > self.cbox.bottom : 
            self.cbox.bottom = target.rect.bottom

        self.offset.x = self.cbox.left - self.borders[">"] # if the cbox doesnt move , offset.x is 0  
        self.offset.y = self.cbox.top - self.borders["^"]  # if the cbox doesnt move , offset.y is 0 

    def custom_draw(self , cbox_target: pygame.sprite.Sprite):

        self.center_camera_box(cbox_target)
        ground_offset = self.ground_rect.topleft - self.offset

        self.display_surface.blit(self.ground_surface , ground_offset)
        for sprite in sorted(self.sprites() , key = lambda sprite: sprite.rect.centery):  
            player_offset = sprite.rect.topleft - self.offset
            self.display_surface.blit(sprite.image , player_offset)  


        
camera_group = Camera()
player = Player(camera_group)  


BLOCK = 100
for _ in range(20): 
    pos_x = random.randint(BLOCK , WIDTH-BLOCK) 
    pos_y = random.randint(BLOCK , HEIGHT-BLOCK)  
    tree = Tree(pos_x , pos_y , camera_group)  



previous_time = time.time()
running = True 
while running:  
    Dt = time.time() - previous_time  
    previous_time = time.time()
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT:

            exit() 
            pygame.quit() 
    
    screen.fill(LIGHT_BLUE)  
    camera_group.update(Dt) 
    camera_group.custom_draw(player)
    pygame.display.update()

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

### Implementation of the Camera with the Keyboard Control 

In [1]:
import math
import pygame 
import random  
import time 

pygame.init()  
WIDTH , HEIGHT = 700 , 700
screen = pygame.display.set_mode((WIDTH , HEIGHT))  
clock = pygame.time.Clock()  
font = pygame.font.Font(None , 30)

def debug(info , y = 10 , x = 10):  
    display_surface = pygame.display.get_surface() 
    debug_surf = font.render(str(info) , True , 'White') 
    debug_rect = debug_surf.get_rect(topleft = (x,y) ) 
    pygame.draw.rect(display_surface , 'Black' , debug_rect)  
    display_surface.blit(debug_surf , debug_rect) 

BLACK = (0,0,0) 
WHITE = (255,255,255) 
RED = (255,0,0) 
GREEN = (0,255,0)  
LIGHT_GREEN = (30,204,102)
PURPLE = (204 , 30 , 102) 
LIGHT_BLUE = (30,102,204) 
DARK_BLUE = (0,0,51) 

class Tree(pygame.sprite.Sprite): 
    def __init__(self, pos_x , pos_y , group) -> None:
        super().__init__(group) 
        self.image = pygame.image.load("img/tree.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 

class Player(pygame.sprite.Sprite): 
    def __init__(self , camera_group ,  pos_x = 350 , pos_y = 350 , speed = 200) -> None:
        super().__init__(camera_group) 
        self.image = pygame.image.load("img/player.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 
        self.direction = pygame.math.Vector2() 
        self.speed = speed 

    def input(self): 
        keys = pygame.key.get_pressed() 
        if keys[pygame.K_UP] : 
            self.direction.y = -1 
        
        elif keys[pygame.K_DOWN] : 
            self.direction.y = 1 

        else : 
            self.direction.y = 0 

        if keys[pygame.K_RIGHT] : 
            self.direction.x = 1 

        elif keys[pygame.K_LEFT]: 
            self.direction.x = -1 

        else : 
            self.direction.x = 0

    def update(self,delta_time) -> None: 
        self.input() 
        movement = self.speed * delta_time  
        debug(self.direction * movement ) 
        self.rect.center +=  self.direction * round(movement) 

class Camera(pygame.sprite.Group): 
    def __init__(self , keyboard_speed = 5) -> None:
        super().__init__()  
        self.display_surface = pygame.display.get_surface()  
        self.ground_surface = pygame.image.load("img/ground.png") 
        self.ground_rect = self.ground_surface.get_rect(topleft = (0,0)) 

        self.offset = pygame.math.Vector2()


        screen_w , screen_h = self.display_surface.get_size() 
        self.borders = {"<":200 , ">" : 200 , "v" : 100 , "^":100} 
        l = self.borders[">"] 
        t = self.borders["^"] 
        w = screen_w - self.borders["<"] - self.borders[">"] 
        h = screen_h - self.borders["v"] - self.borders["^"] 
        self.cbox = pygame.Rect(l , t , w , h)  

        self.keyboard_speed = keyboard_speed

        


    def center_camera_box(self,target: pygame.sprite.Sprite): 
        
        if target.rect.left < self.cbox.left : 
            self.cbox.left = target.rect.left  

        if target.rect.right > self.cbox.right : 
            self.cbox.right = target.rect.right 

        if target.rect.top < self.cbox.top : 
            self.cbox.top = target.rect.top 

        if target.rect.bottom > self.cbox.bottom : 
            self.cbox.bottom = target.rect.bottom

        self.offset.x = self.cbox.left - self.borders[">"] # if the cbox doesnt move , offset.x is 0  
        self.offset.y = self.cbox.top - self.borders["^"]  # if the cbox doesnt move , offset.y is 0  


    def keyboardControl(self , target : pygame.sprite.Sprite): 
        key = pygame.key.get_pressed() 

        """
        WORTH TO MENTION THAT : 

        MOVING CBOX INSTEAD OF INCREASING OFFSETT VALUE IS MUCH MORE RELIABLE METHOD . THE OTHER ONE CAN PRONE TO ERRORS EASILY.
        """



        if key[pygame.K_a]: 
            self.cbox.centerx -= self.keyboard_speed 
        
        if key[pygame.K_d]: 
            self.cbox.centerx += self.keyboard_speed 

        if key[pygame.K_w]: 
            self.cbox.centery -= self.keyboard_speed 

        if key[pygame.K_s]: 
            self.cbox.centery += self.keyboard_speed  

        if key[pygame.K_y]: 
            self.cbox.center = target.rect.center 

        self.offset.x = self.cbox.left - self.borders[">"] # if the cbox doesnt move , offset.x is 0  
        self.offset.y = self.cbox.top - self.borders["^"]  # if the cbox doesnt move , offset.y is 0  

    def custom_draw(self , cbox_target: pygame.sprite.Sprite):

        self.keyboardControl(cbox_target)
        #self.center_camera_box(cbox_target)
        ground_offset = self.ground_rect.topleft - self.offset

        self.display_surface.blit(self.ground_surface , ground_offset)
        for sprite in sorted(self.sprites() , key = lambda sprite: sprite.rect.centery):  
            player_offset = sprite.rect.topleft - self.offset
            self.display_surface.blit(sprite.image , player_offset)  


        
camera_group = Camera()
player = Player(camera_group)  


BLOCK = 100
for _ in range(20): 
    pos_x = random.randint(BLOCK , WIDTH-BLOCK) 
    pos_y = random.randint(BLOCK , HEIGHT-BLOCK)  
    tree = Tree(pos_x , pos_y , camera_group)  



previous_time = time.time()
running = True 
while running:  
    Dt = time.time() - previous_time  
    previous_time = time.time()
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT:

            exit() 
            pygame.quit() 
    
    screen.fill(LIGHT_BLUE)  
    camera_group.update(Dt) 
    camera_group.custom_draw(player)
    pygame.display.update()

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

### Camera with Mouse Control 

<img src="img/camera_mouse_control1.jpg" width="600px">

In this kind of camera , we still have a camera box . But actually there is a major problem implementing this . We can only check for mouse position for 1 frame. The if-else statements that we use to immobilize the position of the mouse , can control it only within 1 frame , so we have to get it from 1 if else , not multiple if elses 

There are 8 points that we have to check. So this means 8 if conditions are gonna be there . 

#### Implementation of camera with mouse control 

In [1]:
import math
import pygame 
import random  
import time 

pygame.init()  
WIDTH , HEIGHT = 700 , 700
screen = pygame.display.set_mode((WIDTH , HEIGHT))  
clock = pygame.time.Clock()  
font = pygame.font.Font(None , 30)

def debug(info , y = 10 , x = 10):  
    display_surface = pygame.display.get_surface() 
    debug_surf = font.render(str(info) , True , 'White') 
    debug_rect = debug_surf.get_rect(topleft = (x,y) ) 
    pygame.draw.rect(display_surface , 'Black' , debug_rect)  
    display_surface.blit(debug_surf , debug_rect) 

BLACK = (0,0,0) 
WHITE = (255,255,255) 
RED = (255,0,0) 
GREEN = (0,255,0)  
LIGHT_GREEN = (30,204,102)
PURPLE = (204 , 30 , 102) 
LIGHT_BLUE = (30,102,204) 
DARK_BLUE = (0,0,51) 

class Tree(pygame.sprite.Sprite): 
    def __init__(self, pos_x , pos_y , group) -> None:
        super().__init__(group) 
        self.image = pygame.image.load("img/tree.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 

class Player(pygame.sprite.Sprite): 
    def __init__(self , camera_group ,  pos_x = 350 , pos_y = 350 , speed = 200) -> None:
        super().__init__(camera_group) 
        self.image = pygame.image.load("img/player.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 
        self.direction = pygame.math.Vector2() 
        self.speed = speed 

    def input(self): 
        keys = pygame.key.get_pressed() 
        if keys[pygame.K_UP] : 
            self.direction.y = -1 
        
        elif keys[pygame.K_DOWN] : 
            self.direction.y = 1 

        else : 
            self.direction.y = 0 

        if keys[pygame.K_RIGHT] : 
            self.direction.x = 1 

        elif keys[pygame.K_LEFT]: 
            self.direction.x = -1 

        else : 
            self.direction.x = 0

    def update(self,delta_time) -> None: 
        self.input() 
        movement = self.speed * delta_time  
        debug(self.direction * movement ) 
        self.rect.center +=  self.direction * round(movement) 

class Camera(pygame.sprite.Group): 
    def __init__(self , keyboard_speed = 5) -> None:
        super().__init__()  
        self.display_surface = pygame.display.get_surface()  
        self.ground_surface = pygame.image.load("img/ground.png") 
        self.ground_rect = self.ground_surface.get_rect(topleft = (0,0)) 

        self.offset = pygame.math.Vector2()


        screen_w , screen_h = self.display_surface.get_size() 
        self.borders = {"<":200 , ">" : 200 , "v" : 100 , "^":100} 
        l = self.borders[">"] 
        t = self.borders["^"] 
        w = screen_w - self.borders["<"] - self.borders[">"] 
        h = screen_h - self.borders["v"] - self.borders["^"] 
        self.cbox = pygame.Rect(l , t , w , h)  

        self.mouse_speed = 0.4 

        


    def center_camera_box(self,target: pygame.sprite.Sprite): 
        
        if target.rect.left < self.cbox.left : 
            self.cbox.left = target.rect.left  

        if target.rect.right > self.cbox.right : 
            self.cbox.right = target.rect.right 

        if target.rect.top < self.cbox.top : 
            self.cbox.top = target.rect.top 

        if target.rect.bottom > self.cbox.bottom : 
            self.cbox.bottom = target.rect.bottom

        self.offset.x = self.cbox.left - self.borders[">"] # if the cbox doesnt move , offset.x is 0  
        self.offset.y = self.cbox.top - self.borders["^"]  # if the cbox doesnt move , offset.y is 0  


    def mouseControl(self): 
        
        mouse = pygame.math.Vector2(pygame.mouse.get_pos())  
        mouse_offset_vector = pygame.math.Vector2()
        lb = self.borders[">"] 
        tb = self.borders["^"] 
        rb = WIDTH - self.borders["<"] 
        bb = HEIGHT - self.borders["v"] 

        if tb < mouse.y < bb :
            if mouse.x < lb :  
                mouse_offset_vector.x = mouse.x - lb 
                pygame.mouse.set_pos((lb , mouse.y))  

            if mouse.x > rb : 
                mouse_offset_vector.x = mouse.x - rb 
                pygame.mouse.set_pos((rb , mouse.y)) 

        elif mouse.y < tb :  

            if mouse.x < lb :  
                mouse_offset_vector = mouse - pygame.math.Vector2((lb , tb ))
                pygame.mouse.set_pos((lb , tb))  

            if mouse.x > rb : 
                mouse_offset_vector = mouse - pygame.math.Vector2((rb ,tb)) 
                pygame.mouse.set_pos((rb , tb))   
                

        elif mouse.y > bb :  
            
            if mouse.x < lb :  
                mouse_offset_vector = mouse - pygame.math.Vector2((lb , bb))
                pygame.mouse.set_pos((lb , bb))  

            if mouse.x > rb : 
                mouse_offset_vector = mouse - pygame.math.Vector2((rb , bb))
                pygame.mouse.set_pos((rb , bb))  

        if lb < mouse.x < rb :
            if mouse.y < tb :  
                mouse_offset_vector.y = mouse.y - tb 
                pygame.mouse.set_pos((mouse.x , tb))  

            if mouse.y > bb : 
                mouse_offset_vector.y = mouse.y - bb 
                pygame.mouse.set_pos((mouse.x , bb)) 

        self.offset += mouse_offset_vector * self.mouse_speed



    def custom_draw(self , cbox_target: pygame.sprite.Sprite):

        
        self.mouseControl()
        ground_offset = self.ground_rect.topleft - self.offset

        self.display_surface.blit(self.ground_surface , ground_offset)
        for sprite in sorted(self.sprites() , key = lambda sprite: sprite.rect.centery):  
            player_offset = sprite.rect.topleft - self.offset
            self.display_surface.blit(sprite.image , player_offset)  


        
camera_group = Camera()
player = Player(camera_group)  


BLOCK = 100
for _ in range(20): 
    pos_x = random.randint(BLOCK , WIDTH-BLOCK) 
    pos_y = random.randint(BLOCK , HEIGHT-BLOCK)  
    tree = Tree(pos_x , pos_y , camera_group)  


pygame.event.set_grab(True)
previous_time = time.time()
running = True 
while running:  
    Dt = time.time() - previous_time  
    previous_time = time.time()
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT:

            exit()  
            pygame.quit()  

        if e.type == pygame.KEYDOWN: 
            if e.key == pygame.K_ESCAPE: 
                pygame.event.post(pygame.event.Event(pygame.QUIT))
    
    screen.fill(LIGHT_BLUE)  
    camera_group.update(Dt) 
    camera_group.custom_draw(player)
    pygame.display.update()

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 

### Zooming Camera Theory 

<img src="img/camera_zoom1.jpg" width="600px">

> We need to add another offset because of the fact that the game starts from internal surfaces 0,0 position . We want it to start from where the player was before  

<img src="img/camera_zoom2.jpg" width="600px">

__IMPORTANT THINGS TO REMEMBER WHEN ZOOMING__ 


> Default game display screen's and internal surface's center must be in the same point. Otherwise when zooming , player moves unintentionaly. 

> Zooming a lot cause bugs. It is recommended if you can limit the zoom scale. 

> offset of the internal surface can be taken with :
>
>` internal_offset.x = (internal_surf_width - screen_width)//2`  
>
>` internal_offset.y = (internal_surf_height - screen_height) // 2` 

### Implementation of Zooming Camera :

In [1]:
import math
from typing import Sequence
import pygame 
import random  
import time 

pygame.init()  
WIDTH , HEIGHT = 700 , 700
screen = pygame.display.set_mode((WIDTH , HEIGHT))  
clock = pygame.time.Clock()  
font = pygame.font.Font(None , 30)

def debug(info , y = 10 , x = 10):  
    display_surface = pygame.display.get_surface() 
    debug_surf = font.render(str(info) , True , 'White') 
    debug_rect = debug_surf.get_rect(topleft = (x,y) ) 
    pygame.draw.rect(display_surface , 'Black' , debug_rect)  
    display_surface.blit(debug_surf , debug_rect) 

BLACK = (0,0,0) 
WHITE = (255,255,255) 
RED = (255,0,0) 
GREEN = (0,255,0)  
LIGHT_GREEN = (30,204,102)
PURPLE = (204 , 30 , 102) 
LIGHT_BLUE = (30,102,204) 
DARK_BLUE = (0,0,51) 

class Tree(pygame.sprite.Sprite): 
    def __init__(self, pos_x , pos_y , group) -> None:
        super().__init__(group) 
        self.image = pygame.image.load("img/tree.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 

class Player(pygame.sprite.Sprite): 
    def __init__(self , camera_group ,  pos_x = 350 , pos_y = 350 , speed = 200) -> None:
        super().__init__(camera_group) 
        self.image = pygame.image.load("img/player.png") 
        self.rect = self.image.get_rect(center = (pos_x , pos_y)) 
        self.direction = pygame.math.Vector2() 
        self.speed = speed 

    def input(self): 
        keys = pygame.key.get_pressed() 
        if keys[pygame.K_UP] : 
            self.direction.y = -1 
        
        elif keys[pygame.K_DOWN] : 
            self.direction.y = 1 

        else : 
            self.direction.y = 0 

        if keys[pygame.K_RIGHT] : 
            self.direction.x = 1 

        elif keys[pygame.K_LEFT]: 
            self.direction.x = -1 

        else : 
            self.direction.x = 0

    def update(self,delta_time) -> None: 
        self.input() 
        movement = self.speed * delta_time  
        debug(self.direction * movement ) 
        self.rect.center +=  self.direction * round(movement) 

class Camera(pygame.sprite.Group): 
    def __init__(self , keyboard_speed = 5) -> None:
        super().__init__()  
        self.display_surface = pygame.display.get_surface()  
        self.ground_surface = pygame.image.load("img/ground.png") 
        self.ground_rect = self.ground_surface.get_rect(topleft = (0,0)) 

        self.offset = pygame.math.Vector2()


        screen_w , screen_h = self.display_surface.get_size() 
        self.borders = {"<":200 , ">" : 200 , "v" : 100 , "^":100} 
        l = self.borders[">"] 
        t = self.borders["^"] 
        w = screen_w - self.borders["<"] - self.borders[">"] 
        h = screen_h - self.borders["v"] - self.borders["^"] 
        self.cbox = pygame.Rect(l , t , w , h)  

        self.keyboard_speed = keyboard_speed 

        # For zooming from keyboard
        self.zoom_scale , self.default_zoom_scale = 1 , 1 
        self.zoom_threshold = 2 
        # Internal Surface 
        self.internal_surf_size = (2500 , 2500) 
        self.i_w , self.i_h = self.internal_surf_size 
        self.internal_surf = pygame.Surface([self.i_w , self.i_h] , pygame.SRCALPHA) 
        self.internal_rect = self.internal_surf.get_rect( center = ((WIDTH)//2 , (HEIGHT)//2))   
        self.internal_surf_vector = pygame.math.Vector2(self.internal_surf_size)  

        self.internal_offset = pygame.math.Vector2() 
        self.internal_offset.x = (self.i_w - WIDTH)//2 
        self.internal_offset.y = (self.i_h - HEIGHT)//2






    def zoom_from_keyboard(self , keys : Sequence): 
 
        if keys[pygame.K_i]: # zoom *I*n 
            
            if  0 < self.zoom_scale:
                self.zoom_scale -= 0.2  
            
            if self.zoom_scale < 0 : 
                self.zoom_scale = 0.1

        if keys[pygame.K_o]: # zoom *O*ut
            if self.zoom_scale < self.zoom_threshold: 
                self.zoom_scale += 0.2  
            else : 
                self.zoom_scale = self.zoom_threshold
        
        if keys[pygame.K_n]: # zoom *N*ormal 
            self.zoom_scale = self.default_zoom_scale


    def keyboardControl(self , target : pygame.sprite.Sprite): 
        key = pygame.key.get_pressed()  

        self.zoom_from_keyboard(keys=key)

        """
        WORTH TO MENTION THAT : 

        MOVING CBOX INSTEAD OF INCREASING OFFSETT VALUE IS MUCH MORE RELIABLE METHOD . THE OTHER ONE CAN PRONE TO ERRORS EASILY.
        """



        if key[pygame.K_a]: 
            self.cbox.centerx -= self.keyboard_speed 
        
        if key[pygame.K_d]: 
            self.cbox.centerx += self.keyboard_speed 

        if key[pygame.K_w]: 
            self.cbox.centery -= self.keyboard_speed 

        if key[pygame.K_s]: 
            self.cbox.centery += self.keyboard_speed  

        if key[pygame.K_y]: 
            self.cbox.center = target.rect.center 

        self.offset.x = self.cbox.left - self.borders[">"] # if the cbox doesnt move , offset.x is 0  
        self.offset.y = self.cbox.top - self.borders["^"]  # if the cbox doesnt move , offset.y is 0  

    def custom_draw(self , cbox_target: pygame.sprite.Sprite):

        
        self.keyboardControl(cbox_target)
        self.internal_surf.fill(LIGHT_BLUE) 

        ground_offset = self.ground_rect.topleft - self.offset + self.internal_offset

        self.internal_surf.blit(self.ground_surface , ground_offset)
        for sprite in sorted(self.sprites() , key = lambda sprite: sprite.rect.centery):  
            player_offset = sprite.rect.topleft - self.offset + self.internal_offset
            self.internal_surf.blit(sprite.image , player_offset)   


        scaled_surf = pygame.transform.scale(self.internal_surf , self.internal_surf_vector *  self.zoom_scale) 
        scaled_rect = scaled_surf.get_rect(center = (WIDTH//2 , HEIGHT//2)) 
        self.display_surface.blit(scaled_surf , scaled_rect)


        
camera_group = Camera()
player = Player(camera_group)  


BLOCK = 100
for _ in range(20): 
    pos_x = random.randint(BLOCK , WIDTH-BLOCK) 
    pos_y = random.randint(BLOCK , HEIGHT-BLOCK)  
    tree = Tree(pos_x , pos_y , camera_group)  



previous_time = time.time()
running = True 
while running:  
    Dt = time.time() - previous_time  
    previous_time = time.time()
    for e in pygame.event.get(): 
        if e.type == pygame.QUIT:

            exit() 
            pygame.quit() 
    
    screen.fill(LIGHT_BLUE)  
    camera_group.update(Dt) 
    camera_group.custom_draw(player)
    pygame.display.update()

pygame 2.1.2 (SDL 2.0.18, Python 3.10.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


error: display Surface quit

: 